In [5]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import time




x_train, x_test = x_train / 255.0, x_test / 255.0

num_classes = 10

def depthwise_separable_conv_block(x, filters, kernel_size, strides):
    # Depthwise Separable Convolution Block
    x = layers.DepthwiseConv2D(kernel_size=kernel_size, strides=strides, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    x = layers.Conv2D(filters, kernel_size=(1, 1), strides=(1, 1), padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    return x

def mobile_net(input_shape):
    # Input Layer
    input_tensor = layers.Input(shape=input_shape)

    # Convolution Block
    x = layers.Conv2D(32, kernel_size=(3, 3), strides=(2, 2), padding='same')(input_tensor)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    # Depthwise Separable Convolution Blocks
    x = depthwise_separable_conv_block(x, 64, (3, 3), (1, 1))
    x = depthwise_separable_conv_block(x, 128, (3, 3), (2, 2))
    x = depthwise_separable_conv_block(x, 128, (3, 3), (1, 1))
    x = depthwise_separable_conv_block(x, 256, (3, 3), (2, 2))
    x = depthwise_separable_conv_block(x, 256, (3, 3), (1, 1))
    x = depthwise_separable_conv_block(x, 512, (3, 3), (2, 2))

    # Global Average Pooling
    x = layers.GlobalAveragePooling2D()(x)

    # Fully Connected Layer
    x = layers.Dense(1024, activation='relu')(x)
    x = layers.Dropout(0.5)(x)

    # Output Layer
    output_tensor = layers.Dense(num_classes, activation='softmax')(x)

    # Model
    model = models.Model(inputs=input_tensor, outputs=output_tensor)

    return model



# Example usage with input shape (28, 28, 1)
input_shape = (28, 28, 1)

# Create the MobileNet-like model
mobile_net_model = mobile_net(input_shape)
mobile_net_model.compile(optimizer='adam',
                   loss='sparse_categorical_crossentropy',
                   metrics=['accuracy'])

# Display model summary
mobile_net_model.summary()

# Train the model with mini-batches of size 32
batch_size = 32

start_time = time.time()


mobile_net_model.fit(x_train, y_train, epochs=15, batch_size=batch_size)

# mobile_net_model.evaluate(x_test.reshape((-1, 28, 28, 1)), y_test)

# Get predicted values for the test set
y_pred_one_hot = mobile_net_model.predict(x_test)

# Convert predictions to integer labels
y_pred_labels = np.argmax(y_pred_one_hot, axis=1)



end_time = time.time()





(32000, 28, 28, 1)
(32000,)
(10000, 28, 28, 1)
(10000,)
(2000, 28, 28, 1)
(2000, 28, 28, 1)
(2000, 28, 28, 1)
Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d_7 (Conv2D)           (None, 14, 14, 32)        320       
                                                                 
 batch_normalization_13 (Ba  (None, 14, 14, 32)        128       
 tchNormalization)                                               
                                                                 
 re_lu_13 (ReLU)             (None, 14, 14, 32)        0         
                                                                 
 depthwise_conv2d_6 (Depthw  (None, 14, 14, 32)        320       
 iseConv2D)                                                      
               

In [4]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.
